# Manejo de datos faltantes

La diferencia entre los datos que se encuentran en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez son limpios y homogéneos.
En particular, a muchos conjuntos de datos interesantes les faltará cierta cantidad de datos.
Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar datos faltantes de diferentes maneras.

En este capítulo, analizaremos algunas consideraciones generales sobre los datos faltantes, veremos cómo Pandas elige representarlos y exploraremos algunas herramientas integradas de Pandas para manejar datos faltantes en Python.
Aquí y a lo largo del libro, me referiré a los datos faltantes en general como valores *nulo*, *NaN* o *NA*.

## Compensaciones en convenciones sobre datos faltantes

Se han desarrollado varios enfoques para rastrear la presencia de datos faltantes en una tabla o "Marco de datos".
Generalmente, giran en torno a una de dos estrategias: usar una *máscara* que indica globalmente los valores faltantes, o elegir un *valor centinela* que indica una entrada faltante.

En el enfoque de enmascaramiento, la máscara podría ser una matriz booleana completamente separada, o podría implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor.

En el enfoque centinela, el valor centinela podría ser alguna convención específica de datos, como indicar un valor entero faltante con –9999 o algún patrón de bits raro, o podría ser una convención más global, como indicar un valor de punto flotante faltante. con `NaN` (No es un número), un valor especial que forma parte de la especificación de punto flotante IEEE.

Ninguno de estos enfoques está exento de compensaciones. El uso de una matriz de máscara separada requiere la asignación de una matriz booleana adicional, lo que agrega una sobrecarga tanto en el almacenamiento como en el cálculo. Un valor centinela reduce el rango de valores válidos que se pueden representar y puede requerir lógica adicional (a menudo no optimizada) en la aritmética de CPU y GPU, porque los valores especiales comunes como "NaN" no están disponibles para todos los tipos de datos.

Como ocurre en la mayoría de los casos en los que no existe una opción universalmente óptima, diferentes lenguajes y sistemas utilizan diferentes convenciones.
Por ejemplo, el lenguaje R utiliza patrones de bits reservados dentro de cada tipo de datos como valores centinela que indican datos faltantes, mientras que el sistema SciDB utiliza un byte adicional adjunto a cada celda para indicar un estado NA.

## Datos faltantes en Pandas

La forma en que Pandas maneja los valores faltantes está limitada por su dependencia del paquete NumPy, que no tiene una noción incorporada de valores NA para tipos de datos que no son de punto flotante.

Quizás Pandas podría haber seguido el ejemplo de R al especificar patrones de bits para cada tipo de datos individual para indicar la nulidad, pero este enfoque resulta bastante difícil de manejar.
Si bien R tiene solo 4 tipos de datos principales, NumPy admite *mucho* más que esto: por ejemplo, mientras que R tiene un solo tipo de entero, NumPy admite 14 tipos de enteros básicos una vez que se tienen en cuenta los anchos de bits disponibles, el signo y el endianismo de la codificación. .
Reservar un patrón de bits específico en todos los tipos NumPy disponibles generaría una cantidad excesiva de gastos generales en varias operaciones especiales para varios tipos, y probablemente incluso requeriría una nueva bifurcación del paquete NumPy. Además, para los tipos de datos más pequeños (como los enteros de 8 bits), sacrificar un bit para usarlo como máscara reduciría significativamente el rango de valores que puede representar.

Debido a estas restricciones y compensaciones, Pandas tiene dos "modos" para almacenar y manipular valores nulos:

- El modo predeterminado es utilizar un esquema de datos faltantes basado en centinela, con valores centinela "NaN" o "Ninguno" según el tipo de datos.
- Alternativamente, puede optar por utilizar los tipos de datos anulables (dtypes) que proporciona Pandas (que se analizan más adelante en este capítulo), lo que da como resultado la creación de una matriz de máscara adjunta para rastrear las entradas faltantes. Estas entradas faltantes se presentan al usuario como el valor especial "pd.NA".

En cualquier caso, las operaciones y manipulaciones de datos proporcionadas por la API de Pandas manejarán y propagarán esas entradas faltantes de manera predecible. Pero para desarrollar cierta intuición sobre *por qué* se toman estas decisiones, profundicemos rápidamente en las compensaciones inherentes a "Ninguno", "NaN" y "NA". Como de costumbre, comenzaremos importando NumPy y Pandas:

In [1]:
import numpy as np
import pandas as pd

### Ninguno como valor centinela

Para algunos tipos de datos, Pandas utiliza "Ninguno" como valor centinela. `None` es un objeto Python, lo que significa que cualquier matriz que contenga `None` debe tener `dtype=object`, es decir, debe ser una secuencia de objetos Python.

Por ejemplo, observe lo que sucede si pasa "Ninguno" a una matriz NumPy:

In [2]:
vals1 = np.array([1, None, 2, 3])
vals1

array([1, None, 2, 3], dtype=object)

Este `dtype=object` significa que la mejor representación de tipo común que NumPy podría inferir para el contenido de la matriz es que son objetos de Python.
La desventaja de usar `Ninguno` de esta manera es que las operaciones sobre los datos se realizarán en el nivel de Python, con mucha más sobrecarga que las operaciones típicamente rápidas que se ven para matrices con tipos nativos:

In [3]:
%timeit np.arange(1E6, dtype=int).sum()

2.73 ms ± 288 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%timeit np.arange(1E6, dtype=object).sum()

92.1 ms ± 3.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Además, debido a que Python no admite operaciones aritméticas con "Ninguno", agregaciones como "suma" o "min" generalmente generarán un error:

In [5]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

Por esta razón, Pandas no utiliza "Ninguno" como centinela en sus matrices numéricas.

### NaN: faltan datos numéricos

El otro centinela de datos faltantes, "NaN", es diferente; es un valor especial de punto flotante reconocido por todos los sistemas que utilizan la representación de punto flotante estándar IEEE:

In [6]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2

array([ 1., nan,  3.,  4.])

Observe que NumPy eligió un tipo de punto flotante nativo para esta matriz: esto significa que, a diferencia de la matriz de objetos anterior, esta matriz admite operaciones rápidas insertadas en código compilado.
Tenga en cuenta que "NaN" es un poco como un virus de datos: infecta cualquier otro objeto que toca.
Independientemente de la operación, el resultado de la aritmética con `NaN` será otro `NaN`:

In [7]:
1 + np.nan

nan

In [8]:
0 * np.nan

nan

Esto significa que los agregados sobre los valores están bien definidos (es decir, no dan lugar a un error) pero no siempre son útiles:

In [9]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

Dicho esto, NumPy proporciona versiones de agregaciones compatibles con ``NaN`` que ignorarán estos valores faltantes:

In [10]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

La principal desventaja de "NaN" es que es específicamente un valor de punto flotante; no existe un valor `NaN` equivalente para números enteros, cadenas u otros tipos.

### NaN y Ninguno en Pandas

`NaN` y `None` tienen su lugar, y Pandas está diseñado para manejarlos dos casi indistintamente, realizando conversiones entre ellos cuando corresponda:

In [11]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Para los tipos que no tienen un valor centinela disponible, Pandas encasilla automáticamente cuando hay valores NA presentes.
Por ejemplo, si establecemos un valor en una matriz de enteros en ``np.nan``, se convertirá automáticamente a un tipo de punto flotante para acomodar el NA:

In [12]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [13]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Observe que además de convertir la matriz de enteros a punto flotante, Pandas convierte automáticamente ``Ninguno`` en un valor ``NaN``.

Si bien este tipo de magia puede parecer un poco pirateada en comparación con el enfoque más unificado de los valores NA en lenguajes de dominios específicos como R, el enfoque centinela/casting de Pandas funciona bastante bien en la práctica y, en mi experiencia, rara vez causa problemas.

La siguiente tabla enumera las convenciones de actualización en Pandas cuando se introducen valores NA:

|Clase de tipo | Conversión al almacenar NA | Valor centinela de NA |
|----------------------|--------------------------------|---- --------------------|
| ``flotante`` | Sin cambios | ``np.nan`` |
| ``objeto`` | Sin cambios | ``Ninguno`` o ``np.nan`` |
| ``entero`` | Transmitir a ``float64`` | ``np.nan`` |
| ``booleano`` | Transmitir a ``objeto`` | ``Ninguno`` o ``np.nan`` |

Tenga en cuenta que en Pandas, los datos de cadena siempre se almacenan con un tipo d ``objeto``.

## Tipos D anulables de Pandas

En las primeras versiones de Pandas, "NaN" y "Ninguno" como valores centinela eran las únicas representaciones de datos faltantes disponibles. La principal dificultad que esto introdujo fue con respecto a la conversión de tipos implícita: por ejemplo, no había manera de representar una matriz entera verdadera con datos faltantes.

Para abordar esta dificultad, Pandas luego agregó *tipos de datos que aceptan valores NULL*, que se distinguen de los tipos de datos normales por el uso de mayúsculas en sus nombres (por ejemplo, `pd.Int32` versus `np.int32`). Para lograr compatibilidad con versiones anteriores, estos tipos de caracteres que admiten valores NULL solo se utilizan si se solicitan específicamente.

Por ejemplo, aquí hay una "Serie" de números enteros con datos faltantes, creada a partir de una lista que contiene los tres marcadores disponibles de datos faltantes:

In [14]:
pd.Series([1, np.nan, 2, None, pd.NA], dtype='Int32')

0       1
1    <NA>
2       2
3    <NA>
4    <NA>
dtype: Int32

Esta representación se puede utilizar indistintamente con las demás en todas las operaciones exploradas en el resto de este capítulo.

## Operando con valores nulos

Como hemos visto, Pandas trata "Ninguno", "NaN" y "NA" como esencialmente intercambiables para indicar valores nulos o faltantes.
Para facilitar esta convención, Pandas proporciona varios métodos para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas.
Ellos son:

- ``isnull``: Genera una máscara booleana que indica los valores faltantes
- ``notnull``: Lo contrario de ``isnull``
- ``dropna``: Devuelve una versión filtrada de los datos
- ``fillna``: Devuelve una copia de los datos con valores faltantes completados o imputados

Concluiremos este capítulo con una breve exploración y demostración de estas rutinas.

### Detección de valores nulos
Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: `isnull` y `notnull`.
Cualquiera de los dos devolverá una máscara booleana sobre los datos. Por ejemplo:

In [15]:
data = pd.Series([1, np.nan, 'hello', None])

In [16]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

Como se menciona en [Indexación y selección de datos](03.02-Indexación-y-Selección-de-datos.ipynb), las máscaras booleanas se pueden usar directamente como un índice de "Serie" o "Marco de datos":

In [17]:
data[data.notnull()]

0        1
2    hello
dtype: object

Los métodos `isnull()` y `notnull()` producen resultados booleanos similares para los objetos ``DataFrame``.

### Eliminación de valores nulos

Además de estos métodos de enmascaramiento, existen los métodos convenientes "dropna".
(que elimina los valores de NA) y `fillna` (que completa los valores de NA). Para una "Serie",
el resultado es sencillo:

In [18]:
data.dropna()

0        1
2    hello
dtype: object

Para un ``DataFrame``, hay más opciones.
Considere el siguiente ``DataFrame``:

In [19]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


No podemos eliminar valores únicos de un "DataFrame"; sólo podemos eliminar filas o columnas enteras.
Dependiendo de la aplicación, es posible que desee uno u otro, por lo que "dropna" incluye varias opciones para un "DataFrame".

De forma predeterminada, `dropna` eliminará todas las filas en las que *cualquier* valor nulo esté presente:

In [20]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternativamente, puede colocar los valores NA a lo largo de un eje diferente. El uso de `axis=1` o `axis='columns'` elimina todas las columnas que contienen un valor nulo:

In [21]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


Pero esto también arroja algunos buenos datos; Es posible que prefiera eliminar filas o columnas con *todos* los valores de NA, o la mayoría de los valores de NA.
Esto se puede especificar a través de los parámetros `how` o `thresh`, que permiten un control preciso del número de nulos que se permiten.

El valor predeterminado es `how='any'`, de modo que se eliminará cualquier fila o columna que contenga un valor nulo.
También puede especificar `how='all'`, que solo eliminará filas/columnas que contengan *todos* valores nulos:

In [22]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [23]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Para un control más detallado, el parámetro `thresh` le permite especificar un número mínimo de valores no nulos para la fila/columna que se mantendrá:

In [24]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


Aquí, la primera y la última fila se han eliminado porque cada una contiene solo dos valores no nulos.

### Llenando valores nulos

A veces, en lugar de eliminar los valores de NA, le gustaría reemplazarlos con un valor válido.
Este valor podría ser un número único como cero, o podría ser algún tipo de imputación o interpolación de los buenos valores.
Podrías hacer esto in situ usando el método `isnull` como máscara, pero debido a que es una operación tan común, Pandas proporciona el método `fillna`, que devuelve una copia de la matriz con los valores nulos reemplazados.

Considere la siguiente "Serie":

In [25]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'), dtype='Int32')
data

a       1
b    <NA>
c       2
d    <NA>
e       3
dtype: Int32

Podemos completar las entradas de NA con un solo valor, como cero:

In [26]:
data.fillna(0)

a    1
b    0
c    2
d    0
e    3
dtype: Int32

Podemos especificar un relleno hacia adelante para propagar el valor anterior hacia adelante:

In [27]:
# forward fill
data.fillna(method='ffill')

a    1
b    1
c    2
d    2
e    3
dtype: Int32

O podemos especificar un relleno hacia atrás para propagar los siguientes valores hacia atrás:

In [28]:
# back fill
data.fillna(method='bfill')

a    1
b    2
c    2
d    3
e    3
dtype: Int32

En el caso de un `DataFrame`, las opciones son similares, pero también podemos especificar un `eje` a lo largo del cual se deben realizar los rellenos:

In [29]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [30]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Tenga en cuenta que si un valor anterior no está disponible durante un llenado directo, el valor NA permanece.